In [ ]:
    @classmethod
    def verify_inverse_kinematics_gradient(cls):
        """
        Verify the analytic gradient of the IK error function using finite differences.
        Angles in RADIANS
        """

        desired_wge = cls._forward_kinematics(np.random.randn(cls.num_joints))

        def scalar_objective(theta):
            wge = cls._forward_kinematics(theta)
            e = tangent_space_error(wge, desired_wge).reshape(-1)
            return 0.5 * (e @ e)

        def analytic_grad(theta):
            wge = cls._forward_kinematics(theta)
            e = tangent_space_error(wge, desired_wge).reshape(-1)
            Jb = cls._body_jacobian(theta)
            return (Jb.T @ e).reshape(-1)

        theta_test = np.random.randn(cls.num_joints)

        g_analytic = analytic_grad(theta_test)
        g_numeric = np.zeros_like(theta_test)
        eps = 1e-6

        for i in range(cls.num_joints):
            d = np.zeros_like(theta_test)
            d[i] = 1.0
            g_numeric[i] = (scalar_objective(theta_test + eps*d) -
                            scalar_objective(theta_test - eps*d)) / (2*eps)

        print("analytic:", g_analytic)
        print("numeric:", g_numeric)
        print("relative error:", np.linalg.norm(g_analytic - g_numeric) /
                                max(1e-8, np.linalg.norm(g_numeric)))